# LLM Workflow Design Patterns: Prompt Chaining
- This file is a project for applying design pattern: prompt-chaining

![Prompt Chaining Diagram](../../assets/prompt_chaining.png)

## Project Idea: Article Summarizer and Quiz Generator

##### Workflow (Prompt Chain)
1. Step 1: Summarize
2. Step 2: Extract Key Terms
3. Step 3: Generate Quiz Questions

In [2]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI

In [72]:
load_dotenv(override=True)

GEMINI_KEY = os.getenv("GOOGLE_API_KEY")

if GEMINI_KEY:
    print("Gemini key loaded successfully")
else:
    print("Failed to load Gemini key.")
    exit(-1)

GEMINI_URL = os.getenv("GOOGLE_API")

if GEMINI_URL:
    print("Gemini URL loaded successfully")
else:
    print("Failed to load Gemini URL.")
    exit(-1)

client =  OpenAI(base_url=GEMINI_URL, api_key=GEMINI_KEY)

Gemini key loaded successfully
Gemini URL loaded successfully


### Step 1: Summarize the text

In [73]:
def summarize(text: str) -> str:
    content = "I will give you a text that you need to summarize, make sure to keep keywords and coherence of the text\n"
    content += f"The text: {text}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    summary = response.choices[0].message.content 
    
    return summary

### Step 2: Extract keywords

In [74]:
import json

def extract_keywords(summary: str) -> list:
    content = f'I will give you a text summary, i need you to extract keywords regarding the topic for a quiz. Be precise and topic-oriented, and ignore general terms. Do not include any markdown. Respond with JSON, and only JSON, with the following format: {{"keywords": [keyword1, keyword2, ...]}}\n'
    content += f"The summary: {summary}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    result = response.choices[0].message.content

    keywords_dict = json.loads(result)

    return keywords_dict["keywords"]


### Step 3: Generate quiz

In [80]:
def generate_quiz(keywords: list, num_questions: int = 10) -> list:
    content = f'I will give you a list of keywords, I want you to generate a {num_questions} question quiz about all keywords ({num_questions} questions total). Do not include any markdown or extra text or text styling. Return the quiz in a list of JSONs format [{{"question": ..., "answer": ...}}]\n'
    content += f"The keywords list: {keywords}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)

    quiz_list = json.loads(response.choices[0].message.content)

    return quiz_list

### Display the quiz

In [81]:
def display_quiz(quiz: list):
    for index, question in enumerate(quiz):
        print(f"Q{index + 1}: {question['question']}")
        print(f"A: {question['answer']}\n")

### Main Program

In [83]:
def main():
    with open("./article.txt", "r", encoding="utf-8") as f:
        article = f.read()

    summary = summarize(article)

    keywords = extract_keywords(summary)

    quiz = generate_quiz(keywords=keywords, num_questions=5)

    display_quiz(quiz)
    

In [ ]:
main()

Q1: What are 'Context engineering' and 'Prompt engineering', and how do they relate to the 'LLM context'?
A: 'Prompt engineering' focuses on crafting effective inputs to guide an LLM's response, while 'Context engineering' is the broader practice of managing and optimizing the entire 'LLM context' (including 'System instructions' and 'Message history') to achieve desired outcomes.

Q2: Explain the relationship between 'Tokens', the 'Context window', and 'LLM constraints'.
A: 'Tokens' are the basic units of text an LLM processes. The 'Context window' defines the maximum number of 'tokens' an LLM can process at once. Exceeding this limit is a primary 'LLM constraint', often leading to truncation or errors.

Q3: How do 'System instructions' and 'Message history' contribute to the 'Dynamic context state' of an LLM?
A: 'System instructions' establish the initial persona or guidelines for the LLM, and 'Message history' records the ongoing dialogue. Both dynamically update the 'LLM context' d